In [1]:
import json
import time

import folium
import numpy as np

from uhull.alpha_shape import get_alpha_shape_polygons

In [2]:
# loggibud dataset download: https://loggibud.s3.amazonaws.com/dataset.zip
file_name_path_list = [
    (
        "Brasília",
        "../maps/points_brasilia_brazil.html",
        "../dataset/data/delivery-instances-1.0/train/df/df-0.json",
    ),
    (
        "Pará",
        "../maps/points_para_brazil.html",
        "../dataset/data/delivery-instances-1.0/train/pa/pa-0.json",
    ),
    (
        "Rio de Janeiro",
        "../maps/points_rio_de_janeiro_brazil.html",
        "../dataset/data/delivery-instances-1.0/train/rj/rj-0.json",
    ),
]

In [3]:
MAP_COLORS = (
    "red",
    "orange",
    "green",
    "purple",
    "pink",
    "darkblue",
    "beige",
    "gray",
    "lightgreen",
    "lightblue",
    "lightgray",
    "cadetblue",
    "black",
)

for location, file_name, file_path in file_name_path_list:
    # Print location
    print(f"\n*** Location = {location} ****")

    # Step 1: load json file
    start_time = time.time()
    with open(file_path) as file:
        data = json.load(file)
    print(
        f"Step 1: Loading json file for {location} took "
        f"{time.time() - start_time:.3f} seconds"
    )

    # Step 2: Create map and draw the points on the map
    start_time = time.time()
    locations = np.array(
        [
            (delivery["point"]["lat"], delivery["point"]["lng"])
            for delivery in data["deliveries"]
        ]
    )
    region_map = folium.Map(
        location=locations.mean(axis=0), zoom_start=11, tiles="cartodbpositron"
    )
    for lat, lng in locations:
        folium.Circle(
            location=(lat, lng),
            radius=20,
            fill=True,
            color="blue",
            popup=f"location = ({lat}, {lng})",
        ).add_to(region_map)
    print(
        f"Step 2: Create map and draw the points on the map for {location} "
        f"took {time.time() - start_time:.3f} seconds."
    )

    # Step 3: Calculate the concave hull for the N location points.
    start_time = time.time()
    polygons = get_alpha_shape_polygons(
        coordinates_points=[(lng, lat) for lat, lng in locations],
        alpha=6 * 1.5,
    )
    print(
        f"Step 3: Calculating the concave hull for the {locations.shape[0]} points "
        f"of {location} took {time.time() - start_time:.3f} seconds."
    )

    # Step 4: Draw the calculated polygons on the map.
    start_time = time.time()
    for i, polygon in enumerate(polygons):
        folium.Polygon(
            locations=[(lat, lng) for lng, lat in polygon],
            color=MAP_COLORS[i % len(MAP_COLORS)],
        ).add_to(region_map)
    print(
        f"Step 4: Drawing the {len(polygons)} calculated polygons on the map took {time.time() - start_time:.3f} seconds"
    )

    # Step 5: Save map as html file
    start_time = time.time()
    region_map.save(file_name)
    print(
        f"Step 5: Saving map as html file took {time.time() - start_time:.3f}  seconds"
    )


*** Location = Brasília ****
Step 1: Loading json file for Brasília took 0.025 seconds
Step 2: Create map and draw the points on the map for Brasília took 0.622 seconds.
Step 3: Calculating the concave hull for the 10436 points of Brasília took 0.805 seconds.
Step 4: Drawing the 22 calculated polygons on the map took 0.001 seconds
Step 5: Saving map as html file took 7.125  seconds

*** Location = Pará ****
Step 1: Loading json file for Pará took 0.033 seconds
Step 2: Create map and draw the points on the map for Pará took 0.415 seconds.
Step 3: Calculating the concave hull for the 4238 points of Pará took 0.319 seconds.
Step 4: Drawing the 11 calculated polygons on the map took 0.000 seconds
Step 5: Saving map as html file took 2.915  seconds

*** Location = Rio de Janeiro ****
Step 1: Loading json file for Rio de Janeiro took 0.041 seconds
Step 2: Create map and draw the points on the map for Rio de Janeiro took 1.735 seconds.
Step 3: Calculating the concave hull for the 26833 point